In [ ]:
import sys

# print('This message will be displayed on the screen.')

# original_stdout = sys.stdout # Save a reference to the original standard output

with open('e9', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
#     print('This message will be written to a file.')
#     sys.stdout = original_stdout
    print(2000, 2000)
    print("S" + "."*1999)
    for i in range(1998):
        print("."*2000)
    print("."*1999 + "G")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................